In [1]:
print('Hi')

Hi


# Modelling

In [2]:
import pandas as pd
import sklearn as sk

In [3]:
known = 3
train = pd.read_csv(f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_train_{known}.csv")
test = pd.read_csv(f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_test_{known}.csv")

In [4]:
train_y = train.loc[:,["output_x","output_y"]]
test_y = test.loc[:,["output_x","output_y"]]
train_x = train.drop(columns=["output_x","output_y"])
test_x = test.drop(columns=["output_x","output_y"])

In [5]:
train_x.head()

,input_0_time,input_0_x,input_0_y,input_1_time,input_1_x,input_1_y,input_2_time,input_2_x,input_2_y
0,22.0,0.0,0.0,21.0,0.0,0.0,11.0,0.1,0.3
1,20.0,0.0,0.0,10.0,-0.4,0.1,1.0,-0.6,0.3
2,30.0,0.0,0.0,19.0,0.0,0.5,11.0,0.2,0.4
3,20.0,0.0,0.0,19.0,0.0,0.0,9.0,0.4,0.0
4,30.0,0.0,0.0,21.0,0.6,0.3,11.0,0.6,0.4


## Baseline Testing

In [6]:
from sklearn.metrics import root_mean_squared_error

In [7]:
dummy_prediction = test_x.loc[:,["input_2_x", "input_2_y"]]
root_mean_squared_error(test_y, dummy_prediction)

np.float64(28.03777274689314)

## Linear Regression

Funktionierende Modelle:
https://scikit-learn.org/1.5/modules/multiclass.html#multioutput-regression

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

linearpipe = Pipeline([("scaler", StandardScaler()), ("regressor", LinearRegression())])
linearpipe.fit(train_x, train_y)

root_mean_squared_error(test_y, linearpipe.predict(test_x))

np.float64(11.249805533936936)

## Neural Network

https://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPRegressor.html

In [9]:
from sklearn.neural_network import MLPRegressor

mlppipe = Pipeline([("scaler", StandardScaler()), ("regressor", MLPRegressor())])
mlppipe.fit(train_x, train_y)

root_mean_squared_error(test_y, mlppipe.predict(test_x))

/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


np.float64(4.584277310262195)

## AutoSklearn   


In [11]:
from autosklearn.regression import AutoSklearnRegressor
automl = AutoSklearnRegressor(
    time_left_for_this_task=120, memory_limit=100*1024, n_jobs=-1, metric=autosklearn.metrics.mean_squared_error
    )
automl.fit(train_x, train_y, dataset_name='diabetes')

ensemble_dict = automl.show_models()
print(ensemble_dict)

TypeError: unhashable type: 'list'